In [9]:
import json
import logging
import os
import sys
import time

import hugectr
from hugectr.inference import CreateInferenceSession
from hugectr.inference import InferenceParams

from mpi4py import MPI
from typing import List, Any

from nvtabular.graph.schema import Schema
from nvtabular.graph.tags import Tags

In [17]:
DATA_ROOT = '/home/jupyter/data/criteo_processed_parquet'

TRAIN_DATA = os.path.join(DATA_ROOT, 'train', '_file_list.txt')
VALID_DATA = os.path.join(DATA_ROOT, 'valid', '_file_list.txt')
SCHEMA_PATH = os.path.join(DATA_ROOT, 'train', 'schema.pbtxt')

MODEL_DIR = '/tmp/saved_model'
CHECKPOINT_DIR = '/tmp/checkpoints'
SNAPSHOT_DIR = '/tmp/snapshots'



In [13]:
def create_model(
    train_data: List[Any],
    valid_data: str,
    slot_size_array: List[Any],
    gpus: List[Any],
    max_eval_batches: int = 300,
    batchsize: int = 2048,
    lr: float = 0.001,
    dropout_rate: float = 0.5,
    workspace_size_per_gpu: float = 61,
    num_dense_features: int = 13,
    num_sparse_features: int = 26,
    nnz_per_slot: int = 2,
    num_workers: int = 12,
    repeat_dataset: bool = True,
):
  """DeepFM Network(https://www.ijcai.org/Proceedings/2017/0239.pdf)."""

  if not gpus:
    gpus = [[0]]

  solver = hugectr.CreateSolver(max_eval_batches=max_eval_batches,
                                batchsize_eval=batchsize,
                                batchsize=batchsize,
                                lr=lr,
                                vvgpu=gpus,
                                repeat_dataset=repeat_dataset,
                                i64_input_key=True)

  reader = hugectr.DataReaderParams(
      data_reader_type=hugectr.DataReaderType_t.Parquet,
      source=train_data,
      eval_source=valid_data,
      slot_size_array=slot_size_array,
      check_type=hugectr.Check_t.Non,
      num_workers=num_workers)

  optimizer = hugectr.CreateOptimizer(optimizer_type=hugectr.Optimizer_t.Adam,
                                     update_type=hugectr.Update_t.Global,
                                     beta1=0.9,
                                     beta2=0.999,
                                     epsilon=0.0000001)

  model = hugectr.Model(solver, reader, optimizer)

  model.add(hugectr.Input(
      label_dim=1, label_name="label",
      dense_dim=num_dense_features, dense_name="dense",
      data_reader_sparse_param_array=[hugectr.DataReaderSparseParam(
          "data1", nnz_per_slot, False, num_sparse_features)]))

  model.add(hugectr.SparseEmbedding(
      embedding_type=hugectr.Embedding_t.LocalizedSlotSparseEmbeddingHash,
      workspace_size_per_gpu_in_mb=workspace_size_per_gpu,
      embedding_vec_size=16,
      combiner="sum",
      sparse_embedding_name="sparse_embedding1",
      bottom_name="data1",
      slot_size_array=slot_size_array,
      optimizer=optimizer))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Reshape,
                               bottom_names=["sparse_embedding1"],
                               top_names=["reshape1"],
                               leading_dim=11))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Slice,
                               bottom_names=["reshape1"],
                               top_names=["slice11", "slice12"],
                               ranges=[(0, 10), (10, 11)]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Reshape,
                               bottom_names=["slice11"],
                               top_names=["reshape2"],
                               leading_dim=260))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Reshape,
                               bottom_names=["slice12"],
                               top_names=["reshape3"],
                               leading_dim=26))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Slice,
                               bottom_names=["dense"],
                               top_names=["slice21", "slice22"],
                               ranges=[(0, 13), (0, 13)]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.WeightMultiply,
                               bottom_names=["slice21"],
                               top_names=["weight_multiply1"],
                               weight_dims=[13, 10]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.WeightMultiply,
                               bottom_names=["slice22"],
                               top_names=["weight_multiply2"],
                               weight_dims=[13, 1]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Concat,
                               bottom_names=["reshape2", "weight_multiply1"],
                               top_names=["concat1"]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Slice,
                               bottom_names=["concat1"],
                               top_names=["slice31", "slice32"],
                               ranges=[(0, 390), (0, 390)]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.InnerProduct,
                               bottom_names=["slice31"],
                               top_names=["fc1"],
                               num_output=400))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.ReLU,
                               bottom_names=["fc1"],
                               top_names=["relu1"]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Dropout,
                               bottom_names=["relu1"],
                               top_names=["dropout1"],
                               dropout_rate=dropout_rate))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.InnerProduct,
                               bottom_names=["dropout1"],
                               top_names=["fc2"],
                               num_output=400))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.ReLU,
                               bottom_names=["fc2"],
                               top_names=["relu2"]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Dropout,
                               bottom_names=["relu2"],
                               top_names=["dropout2"],
                               dropout_rate=dropout_rate))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.InnerProduct,
                               bottom_names=["dropout2"],
                               top_names=["fc3"],
                               num_output=400))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.ReLU,
                               bottom_names=["fc3"],
                               top_names=["relu3"]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Dropout,
                               bottom_names=["relu3"],
                               top_names=["dropout3"],
                               dropout_rate=dropout_rate))
  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.InnerProduct,
                               bottom_names=["dropout3"],
                               top_names=["fc4"],
                               num_output=1))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.FmOrder2,
                               bottom_names=["slice32"],
                               top_names=["fmorder2"],
                               out_dim=10))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.ReduceSum,
                               bottom_names=["fmorder2"],
                               top_names=["reducesum1"],
                               axis=1))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.Concat,
                               bottom_names=["reshape3", "weight_multiply2"],
                               top_names=["concat2"]))

  model.add(hugectr.DenseLayer(layer_type=hugectr.Layer_t.ReduceSum,
                               bottom_names=["concat2"],
                               top_names=["reducesum2"],
                               axis=1))

  model.add(hugectr.DenseLayer(
      layer_type=hugectr.Layer_t.Add,
      bottom_names=["fc4", "reducesum1", "reducesum2"],
      top_names=["add"]))

  model.add(hugectr.DenseLayer(
      layer_type=hugectr.Layer_t.BinaryCrossEntropyLoss,
      bottom_names=["add", "label"],
      top_names=["loss"]))

  model.compile()

  return model


In [14]:
def retrieve_cardinalities(schema_path):
  """Retrieves cardinalities from schema."""

  schema = Schema.load(schema_path)
  cardinalities = {
      key: value.properties['embedding_sizes']['cardinality']
      for key, value in schema.column_schemas.items()
      if Tags.CATEGORICAL in value.tags
  }

  return cardinalities

In [21]:
cardinalities = retrieve_cardinalities(SCHEMA_PATH)
slot_size_array = [int(cardinality)
                   for cardinality in cardinalities.values()]
slot_size_array

[18792578,
 35176,
 17091,
 7383,
 20154,
 4,
 7075,
 1403,
 63,
 12687136,
 1054830,
 297377,
 11,
 2209,
 10933,
 113,
 4,
 972,
 15,
 19550853,
 5602712,
 16779972,
 375290,
 12292,
 101,
 35]

In [22]:
NUM_EPOCHS = 0
MAX_ITERATIONS = 25000
EVAL_INTERVAL = 1000
EVAL_BATCHES = 500
EVAL_BATCHES_FINAL = 2500
DISPLAY_INTERVAL = 200
SNAPSHOT_INTERVAL = 0
PER_GPU_BATCH_SIZE = 2048
LR = 0.001
DROPOUT_RATE = 0.5
NUM_WORKERS = 12

GPUS = [[0,1]]

model = create_model(
    train_data=[TRAIN_DATA],
    valid_data=VALID_DATA,
    slot_size_array=slot_size_array,
    gpus=GPUS,
    max_eval_batches=EVAL_BATCHES,
    batchsize=PER_GPU_BATCH_SIZE*len(GPUS[0]),
    lr=LR,
    dropout_rate=DROPOUT_RATE,
    num_workers=NUM_WORKERS)
    

HugeCTR Version: 3.3
====================================================Model Init=====================================================
[HUGECTR][20:47:29][WARNING][RANK0]: The model name is not specified when creating the solver.
[HUGECTR][20:47:30][INFO][RANK0]: Global seed is 360504220


RuntimeError: Runtime error: OS call failed or operation not supported on this OS
	cudaGetDeviceCount(&dev_count) at ResourceManagerCore(/hugectr/HugeCTR/src/resource_managers/resource_manager_core.cpp:135)